In [127]:
import numpy as pd
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time
import seaborn as sns
import re
import math
sns.set_style("whitegrid")
#sns.set(style="darkgrid")
sns.set_palette("tab10")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import datetime as dt
params = {'legend.fontsize':15, 
          'legend.title_fontsize':16,
          'figure.figsize':(15,5),
          'axes.labelsize': 18,
          'axes.titlesize':20,
          'xtick.labelsize':18,
          'ytick.labelsize':18}
plt.rcParams.update(params)
img_fmt = 'svg'
import warnings
warnings.filterwarnings("ignore")

In [128]:
juegos = pd.read_csv('data.csv')
juegos.head()

,id,name,released,rating,platforms,developers,genres,esrb_rating
0,388309,Diablo IV,NaN,5.00,Xbox One||PlayStation 4||PC,Activision Blizzard,Action||RPG,NaN
1,132,The World Ends with You: Solo Remix,8/27/2012,4.86,iOS,Square Enix,Action||RPG,Everyone 10+
2,229144,Hide Online,NaN,4.83,Android,HitRock Games,Action,NaN
3,257281,Kara no Shojo,10/22/2018,4.83,PC,Innocent Grey,Adventure,Adults Only
4,53135,Astro Boy: Omega Factor,12/18/2003,4.83,Game Boy Advance,Hitmaker||Treasure,NaN,NaN


In [129]:
juegos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           1499 non-null   int64  
 1   name         1499 non-null   object 
 2   released     1473 non-null   object 
 3   rating       1499 non-null   float64
 4   platforms    1499 non-null   object 
 5   developers   1425 non-null   object 
 6   genres       1383 non-null   object 
 7   esrb_rating  642 non-null    object 
dtypes: float64(1), int64(1), object(6)
memory usage: 93.8+ KB


In [130]:
juegos.isna().sum()

id               0
name             0
released        26
rating           0
platforms        0
developers      74
genres         116
esrb_rating    857
dtype: int64

In [131]:
juegos['released'] = juegos['released'].fillna(juegos['released'].mode()[0])
juegos['developers'] = juegos['developers'].fillna(juegos['developers'].mode()[0])
juegos['genres'] = juegos['genres'].fillna(juegos['genres'].mode()[0])
juegos['esrb_rating'] = juegos['esrb_rating'].fillna(juegos['esrb_rating'].mode()[0])

In [132]:
juegos.isna().sum()

id             0
name           0
released       0
rating         0
platforms      0
developers     0
genres         0
esrb_rating    0
dtype: int64

In [133]:
juegos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1499 entries, 0 to 1498
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           1499 non-null   int64  
 1   name         1499 non-null   object 
 2   released     1499 non-null   object 
 3   rating       1499 non-null   float64
 4   platforms    1499 non-null   object 
 5   developers   1499 non-null   object 
 6   genres       1499 non-null   object 
 7   esrb_rating  1499 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 93.8+ KB


In [134]:
juegos['genres'] = juegos['genres'].apply((lambda x: x.split(",")[0]))
juegos['genres'].head()

0    Action||RPG
1    Action||RPG
2         Action
3      Adventure
4       Strategy
Name: genres, dtype: object

In [135]:
count = juegos['genres'].value_counts()
percent = 100*juegos['genres'].value_counts(normalize=True)
pd.DataFrame({'count':count, 'percent':percent.round(1)})

,count,percent
genres,,
Strategy,234,15.6
RPG,112,7.5
Action,97,6.5
Action||Adventure,83,5.5
Action||Shooter,68,4.5
...,...,...
Adventure||Strategy||Indie,1,0.1
Action||Adventure||RPG||Indie||Platformer,1,0.1
Adventure||Family||Indie,1,0.1


In [136]:
count = juegos['esrb_rating'].value_counts()
percent = 100*juegos['esrb_rating'].value_counts(normalize=True)
pd.DataFrame({'count':count, 'percent':percent.round(1)})

,count,percent
esrb_rating,,
Mature,1077,71.8
Teen,217,14.5
Everyone,93,6.2
Everyone 10+,88,5.9
Adults Only,20,1.3
Rating Pending,4,0.3


In [137]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

In [138]:
juegos['genres'] = juegos['genres'].fillna('')
tfidf_matrix = tfidf.fit_transform(juegos['genres'])
tfidf_matrix.shape

(1499, 21)

In [139]:
from sklearn.metrics.pairwise import linear_kernel

cosire_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [140]:
indices = pd.Series(juegos.index,index = juegos['name']).drop_duplicates()

In [141]:
def get_Recomendations(name, cosine_sim = cosire_sim):
    idx = indices[name]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]
    juego_indices = [i[0]for i in sim_scores]
    return juegos['name'].iloc[juego_indices]


In [142]:
get_Recomendations('Hide Online')

9       METAL GEAR SOLID V: THE DEFINITIVE EXPERIENCE
36     Metal Gear Rising: Revengeance - Jetstream Sam
42                                 Drill Dozer (2005)
43                             Jackie Chan Adventures
55                                             Eroico
60                            SCP: Containment Breach
80          ZONE OF THE ENDERS: The 2nd Runner - M∀RS
83                                       Samurai Gunn
93                                  Super Bomberman 3
106           Shinobi III: Return of the Ninja Master
Name: name, dtype: object

In [161]:
features = ['platforms', 'developers', 'esrb_rating', 'genres', 'name']
filters = juegos[features]

def clean_data(x):
    return str.lower(x.replace(" ",""))

for feature in features:
    filters[feature] = filters[feature].apply(clean_data)

filters.head()

,platforms,developers,esrb_rating,genres,name
0,xboxone||playstation4||pc,activisionblizzard,mature,action||rpg,diabloiv
1,ios,squareenix,everyone10+,action||rpg,theworldendswithyou:soloremix
2,android,hitrockgames,mature,action,hideonline
3,pc,innocentgrey,adultsonly,adventure,karanoshojo
4,gameboyadvance,hitmaker||treasure,mature,strategy,astroboy:omegafactor


In [164]:
def create_soup(x):
    return x['platforms'] + ' ' + x['developers'] + ' ' + x['genres']+ ' ' + x['esrb_rating']

filters['soup'] = filters.apply(create_soup, axis = 1)

In [165]:
from sklearn.feature_extraction.text import CountVectorizer

count =  CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(filters['soup'])

In [166]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [167]:
filters.head()

,platforms,developers,esrb_rating,genres,name,soup
0,xboxone||playstation4||pc,activisionblizzard,mature,action||rpg,diabloiv,xboxone||playstation4||pc activisionblizzard a...
1,ios,squareenix,everyone10+,action||rpg,theworldendswithyou:soloremix,ios squareenix action||rpg everyone10+
2,android,hitrockgames,mature,action,hideonline,android hitrockgames action mature
3,pc,innocentgrey,adultsonly,adventure,karanoshojo,pc innocentgrey adventure adultsonly
4,gameboyadvance,hitmaker||treasure,mature,strategy,astroboy:omegafactor,gameboyadvance hitmaker||treasure strategy mature


In [168]:
filters = filters.reset_index()
indices = pd.Series(filters.index, index=filters['name'])

In [169]:
def get_better_Recomendations(name, cosine_sim = cosire_sim):
    name = name.replace(' ', '').lower()
    idx = indices[name]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores= sorted(sim_scores, key=lambda x: x[1], reverse= True)
    sim_scores = sim_scores[1:11]
    game_indices = [i[0] for i in sim_scores]
    return juegos['name'].iloc[game_indices]

In [177]:
get_better_Recomendations('The Legend of Zelda: Breath of the Wild', cosine_sim2)

570     The Legend of Zelda: Link's Awakening (2019)
432                                         Pikmin 3
126        The Legend of Zelda: Twilight Princess HD
504         The Legend of Zelda: Collector's Edition
666                             Super Mario 3D World
1319                                      Splatoon 2
426             The Legend of Zelda: Ocarina of Time
197          The Legend of Zelda: Ocarina of Time 3D
344                            Metroid: Zero Mission
823                               Ring Fit Adventure
Name: name, dtype: object